In [24]:
from IPython.display import Math
import math

# Design Requirements
# External parallel FET shunt dimming
fSW = 500000 # Hz Switching frequency (at nominal VIN, VO)
VIN = 48 # V Nominal input voltage:
VIN_MAX = 75 # V Maximum input voltage
VO = 36 # V Nominal output voltage (number of LEDs x forward voltage)
rD = 0.02 # Ohm LED string dynamic resistance
ILED = 1 # A Average LED current
ΔiLED_PP = ΔiL_PP = 0.1 # A LED current ripple, Inductor current ripple
ΔvIN_PP = 2.0 # V Input voltage ripple
VTURN_ON = 34 # V UVLO characteristics
VHYS = 2.0 # V UVLO characteristics
η = 0.95 # Expected efficiency

D = VO/VIN/η
print("D =", D, "")

D = 0.7894736842105263 


In [25]:
# Nominal Switching Frequency
# Assume COFF = 470 pF
COFF = 470e-12
ROFF = -(1-VO/(η*VIN)) / ((COFF + 20e-12)*fSW*math.log(1-1.24/VO))
print("ROFF =", ROFF/1e3, "kΩ, use 1% tol")
ROFF1 = 24.3e3
del ROFF
print("Closest 1% ROFF1 =", ROFF1/1e3, "kΩ")

# actual tOFF and target fSW are:
tOFF = -(COFF + 20e-12) * ROFF1 * math.log(1-1.24/VO)
fSW = (1-VO/(η*VIN))/tOFF
print("tOFF =", tOFF*1e9, "ns")
print("fSW =", fSW/1e3, "kHz")


print('\nInductor Ripple Current')
# Inductor Ripple Current
L = VO*tOFF/ΔiL_PP
print("L =", L*1e6, "μH")
L1 = 150.0e-6
del L
print("Closest standard value is", L1*1e6, "μH")
# the actual ΔiL-PP is:
ΔiL_PP = VO*tOFF/L1
print("actual ΔiL-PP is", ΔiL_PP, "A")


print('\nAverage LED Current')
# Average LED Current
IL_MAX = ILED+ΔiL_PP/2
print("IL_MAX", IL_MAX, "A")
# Assume VADJ = 1.24 V and solve for RSNS:
VADJ = 1.24
RSNS = VADJ/(5*IL_MAX)
print("RSNS", RSNS, "Ω")
del RSNS
RSNS = 0.470
RSNS1 = RSNS**2 / (2*RSNS)
print("Closest 1% RSNS1 =", RSNS1, "Ω")
ILED = VADJ/(5*RSNS1)-ΔiL_PP/2
print("actual ILED is", ILED, "A")


print('\nInput Capacitance')
# Input Capacitance
tON = 1/fSW - tOFF
CIN_MIN = ILED*tON/ΔvIN_PP
CIN = CIN_MIN*2
IIN_RMS = ILED * fSW * math.sqrt(tON*tOFF)
print("tON =", tON*1e9, "ns")
print("CIN ≥", CIN*1e6, "μF")
print("IIN_RMS =", IIN_RMS*1e3, "mA")


print('\nPFET')
# PFET
IT = VO*ILED/VIN/η
# FET current
print("IT =", IT*1e3, "mA")
# A 70V, 5.7 A PFET is chosen with RDS-ON = 190 mΩ and Qg = 20 nC. Determine IT-RMS and PT:
# Try DMPH6050SK3, 60V 20A PFET with RDS-On = 50 mΩ and Qg = 12-25 nC
RDSON = 0.049
IT_RMS = ILED*math.sqrt(1*(1+1/12*(ΔiL_PP/ILED)**2))
PT = IT_RMS**2 * RDSON
print("IT_RMS =", IT_RMS*1e3, "mA")
print("PT =", PT*1e3, "mW")


print('\nDiode')
# Schottky Diode. Higher current diodes with a lower forward voltage are suggested
VD = 0.850 # V, chosen diode S310FA, forward voltage (D1->3A, 100V, SMC)
ID = (1-D)*ILED
PD = ID*VD
print("ID =", ID*1e3, "mA")
print("PD =", PD*1e3, "mW")

print('\nOff Time Resistor')
VDD = 12 # V, secondary power source
ROFF2 = ROFF1*VDD/ILED/RDSON
print("ROFF1 =", ROFF1/1e3, "kΩ")
print("ROFF2 =", ROFF2/1e3, "kΩ")
ROFF2 = 4.42e6
print("Closest 1% ROFF2 =", ROFF2/1e3, "kΩ")


print('\nUVLO')
RUV2 = VHYS/22e-6
print("RUV2 =", RUV2/1e3, "kΩ")
RUV2 = 91e3
print("Closest 1% RUV2 =", RUV2/1e3, "kΩ")
VHYS = RUV2*22e-6
print("So VHYS =", VHYS, "V")
RUV1 = 1.24*RUV2/(VTURN_ON-1.24)
print("RUV1 =", RUV1/1e3, "kΩ")
RUV1 = 3.48e3
print("Closest 1% RUV1 =", RUV1/1e3, "kΩ")
VTURN_ON = 1.24*(RUV1+RUV2)/RUV1
print("So VTURN_ON =", VTURN_ON, "V")

ROFF = 24.51500507367445 kΩ, use 1% tol
Closest 1% ROFF1 = 24.3 kΩ
tOFF = 417.35985436754584 ns
fSW = 504.42397270935083 kHz

Inductor Ripple Current
L = 150.2495475723165 μH
Closest standard value is 150.0 μH
actual ΔiL-PP is 0.100166365048211 A

Average LED Current
IL_MAX 1.0500831825241055 A
RSNS 0.23617176632033812 Ω
Closest 1% RSNS1 = 0.235 Ω
actual ILED is 1.005235966412065 A

Input Capacitance
tON = 1565.0994538782982 ns
CIN ≥ 1.5732942620503458 μF
IIN_RMS = 409.8170691497521 mA

PFET
IT = 793.607341904262 mA
IT_RMS = 1005.651757100329 mA
PT = 49.55543737138997 mW

Diode
ID = 211.62862450780312 mA
PD = 179.88433083163267 mW

Off Time Resistor
ROFF1 = 24.3 kΩ
ROFF2 = 5920.023364667229 kΩ
Closest 1% ROFF2 = 4420.0 kΩ

UVLO
RUV2 = 90.9090909090909 kΩ
Closest 1% RUV2 = 91.0 kΩ
So VHYS = 2.002 V
RUV1 = 3.4444444444444446 kΩ
Closest 1% RUV1 = 3.48 kΩ
So VTURN_ON = 33.66528735632184 V
